In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#constants
L= 128              #lattice size
N=L*L             #no. of spins
J=1               #coupling constant
dT=0.05           #increment in temp
EQRUN=50000       #equilibrium steps
RUN=100000        #sampling steps

#variables 
s=[0] * N               #spin values
nn=[0] * (4*N)          #list of nearest neighbours
p_accept=[0] * 4        #accepectance probability
M =0                    #magnetization

In [4]:
def initialize():               #initializing spin configuration
    for i in range(N):
        s[i]=1
        M +=s[i]

In [5]:
def set_weights(T):             #list of possible acceptance proababilities 
    p_accept[0]= np.e**(8.0*J/T)
    p_accept[1]= np.e**(4.0*J/T)
    p_accept[2]= np.e**(-4.0*J/T)
    p_accept[3]= np.e**(-8.0*J/T)
    

In [6]:
def nnlist():       #Make neighbour list of spins
    idx=0
    idxp=0
    ip=0
    jp=0

    for i in range(L):
        for j in range(L):
            idx = i * L + j
            jp = 0 if (j+1) == L else (j+1)		     # pbc for righmost spins
            idxp = i * L + jp
            nn[idx * 4 + 0] = idxp 
            jp = L-1 if j-1 == -1 else j-1 		     # pbc for leftmost spins
            idxp = i * L + jp
            nn[idx * 4 + 1] = idxp
            ip = L-1 if i-1 == -1 else i-1 		     # pbc for bottom-most spins
            idxp = ip * L + j
            nn[idx * 4 + 2] = idxp
            ip = 0 if i+1 == L else i+1 		     # pbc for topmost spins
            idxp = ip * L + j
            nn[idx * 4 + 3] = idxp
    

In [7]:
def mc(T):         #monte carlo moves
    for steps in range(N):
        i = int(N * (np.random.uniform(0,1)))
        ss=0
        M=0
        for i in range(N):
            s[i]=1
            M +=s[i]
        nnlist()
        for n in range(4):
            ss+= s[nn[(i * 4 + n)]]
        ss*=s[i]
        
        p = 0
        set_weights(T)
        
        if(ss == -4):
            p = p_accept[0]
        elif(ss == -2):
            p = p_accept[1]
        elif(ss == 2):
            p = p_accept[2]
        elif(ss == 4):
            p = p_accept[3]
        r = np.random.uniform(0,1)

        if r<p :
            s[i] *= -1
            M+= 2*s[i] 

In [ ]:
for i in range(N):
        s[i]=1
        M +=s[i]
nnlist()
T=1.0
for t in range(40):
    set_weights(T)
    for i in range(EQRUN):
        mc(T)
        if (t/10000 == 1):
            print("Equilibration: T=",T,"t=",i,"\n")
    M_av=0
    M2_av=0
    M4_av=0
    for i in range(RUN):
        mc(T)
        m = (M/N)
        M_av += m
        M2_av += m * m
        M4_av += m * m * m * m
        print("Sampling: T= ",T,"t=",i,"\n")
    M_av/= RUN
    M2_av/= RUN
    M4_av/= RUN
    chi = M2_av - M_av * M_av
    U4 = 1 - M4_av / (3 * M2_av * M2_av)
    T += dT
    
    
        